In [ ]:
# CREDIT: github.com/mmamica/emotion-recognition/

import os
import sys
import shutil
import numpy as np
sys.path.insert(0, '../scripts')

from eeg_cnn_lib import transfer_all_class, prepare_data_labels
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras import optimizers

# The images are in a folder named 'shapes/training'
train_dir_name = '../../../data/classification/arousal/train'
validation_dir_name = '../../../data/classification/arousal/validation'
test_dir_name = '../../../data/classification/arousal/test'


# All images are 32 pixels
img_size = (32,32)
img_size_rgba = (32,32,4)

# The folder contains a subfolder for each class of shape
classes = sorted(os.listdir(train_dir_name))

Using TensorFlow backend.


In [ ]:
transfer_all_class(test_dir_name,train_dir_name,1.0,classes)
transfer_all_class(validation_dir_name,train_dir_name,1.0,classes)
transfer_all_class(train_dir_name,test_dir_name,0.2,classes)

In [ ]:
X, Y = prepare_data_labels(classes, train_dir_name)
X=np.asarray(X)
Y=np.asarray(Y)

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), strides=1, padding='same', input_shape=img_size_rgba, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Conv2D(64, (3, 3), strides=1, padding='same', input_shape=img_size_rgba, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Conv2D(128, (3, 3), strides=1, padding='same', input_shape=img_size_rgba, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=2))

model.add(Flatten())

model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))

model.add(Dense(len(classes), activation='softmax'))

opt = optimizers.Adam(lr=0.001)

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
kf = KFold(n_splits=10, shuffle=False)
accuracies = []

In [ ]:
for train_index, val_index in kf.split(X, Y):
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    transfer_all_class(validation_dir_name,train_dir_name,1.0,classes)
    for index in range(len(X_val)):
        class_label=''
        if(Y_val[index]==0):
            class_label=classes[0]
        else:
            class_label=classes[1]  
        #Then, copy the validation images to the validation folder
        shutil.move(train_dir_name+"/"+class_label+'/'+X_val[index], 
                    validation_dir_name+"/"+class_label+'/'+X_val[index])
        
    batch_size = 256
    epoch=100
    datagen = ImageDataGenerator(rescale=1./255)

    train_generator = datagen.flow_from_directory(
        train_dir_name,
        target_size=img_size,
        color_mode='rgba',
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = datagen.flow_from_directory(
        validation_dir_name,
        target_size=img_size,
        color_mode='rgba',
        batch_size=batch_size,
        class_mode=None)
        
    history=model.fit_generator(train_generator, 
                        epochs=epoch)
        
    predictions = model.predict_generator(validation_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    y_true = validation_generator.classes
    accuracy=accuracy_score(y_true, y_predictions)
    accuracies.append(accuracy)
    print(accuracy)

Found 12042 images belonging to 2 classes.
Found 1338 images belonging to 2 classes.
Epoch 1/100
48/48 [==============================] - 9s 194ms/step - loss: 0.6939 - accuracy: 0.5354
Epoch 2/100
48/48 [==============================] - 8s 162ms/step - loss: 0.6899 - accuracy: 0.5363
Epoch 3/100
48/48 [==============================] - 8s 159ms/step - loss: 0.6904 - accuracy: 0.5411
Epoch 4/100
48/48 [==============================] - 8s 158ms/step - loss: 0.6902 - accuracy: 0.5372
Epoch 5/100
48/48 [==============================] - 8s 158ms/step - loss: 0.6898 - accuracy: 0.5383
Epoch 6/100
48/48 [==============================] - 8s 160ms/step - loss: 0.6888 - accuracy: 0.5375
Epoch 7/100
48/48 [==============================] - 8s 158ms/step - loss: 0.6899 - accuracy: 0.5409
Epoch 8/100
48/48 [==============================] - 8s 158ms/step - loss: 0.6892 - accuracy: 0.5412
Epoch 9/100
48/48 [==============================] - 8s 158ms/step - loss: 0.6886 - accuracy: 0.5409
Epoch 

48/48 [==============================] - 8s 157ms/step - loss: 0.3133 - accuracy: 0.8440
Epoch 81/100
48/48 [==============================] - 7s 155ms/step - loss: 0.3895 - accuracy: 0.8132
Epoch 82/100
48/48 [==============================] - 7s 156ms/step - loss: 0.3519 - accuracy: 0.8316
Epoch 83/100
48/48 [==============================] - 7s 155ms/step - loss: 0.3510 - accuracy: 0.8272
Epoch 84/100
48/48 [==============================] - 7s 156ms/step - loss: 0.3088 - accuracy: 0.8533
Epoch 85/100
48/48 [==============================] - 8s 159ms/step - loss: 0.3142 - accuracy: 0.8406
Epoch 86/100
48/48 [==============================] - 7s 155ms/step - loss: 0.3053 - accuracy: 0.8540
Epoch 87/100
48/48 [==============================] - 7s 155ms/step - loss: 0.2982 - accuracy: 0.8579
Epoch 88/100
48/48 [==============================] - 7s 155ms/step - loss: 0.3014 - accuracy: 0.8511
Epoch 89/100
48/48 [==============================] - 7s 156ms/step - loss: 0.3404 - accuracy: 

48/48 [==============================] - 7s 155ms/step - loss: 0.1489 - accuracy: 0.9328
Epoch 60/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1484 - accuracy: 0.9329
Epoch 61/100
48/48 [==============================] - 7s 156ms/step - loss: 0.1603 - accuracy: 0.9283
Epoch 62/100
48/48 [==============================] - 7s 156ms/step - loss: 0.1459 - accuracy: 0.9346
Epoch 63/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1484 - accuracy: 0.9311
Epoch 64/100
48/48 [==============================] - 7s 156ms/step - loss: 0.1515 - accuracy: 0.9312
Epoch 65/100
48/48 [==============================] - 7s 156ms/step - loss: 0.1380 - accuracy: 0.9368
Epoch 66/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1446 - accuracy: 0.9339
Epoch 67/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1550 - accuracy: 0.9300
Epoch 68/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1403 - accuracy: 

Epoch 38/100
48/48 [==============================] - 7s 156ms/step - loss: 0.1043 - accuracy: 0.9556
Epoch 39/100
48/48 [==============================] - 7s 156ms/step - loss: 0.1181 - accuracy: 0.9507
Epoch 40/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1421 - accuracy: 0.9394
Epoch 41/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1311 - accuracy: 0.9402
Epoch 42/100
48/48 [==============================] - 7s 156ms/step - loss: 0.1270 - accuracy: 0.9447
Epoch 43/100
48/48 [==============================] - 7s 156ms/step - loss: 0.1105 - accuracy: 0.9504
Epoch 44/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1003 - accuracy: 0.9545
Epoch 45/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1028 - accuracy: 0.9537
Epoch 46/100
48/48 [==============================] - 8s 156ms/step - loss: 0.1006 - accuracy: 0.9527
Epoch 47/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1008

48/48 [==============================] - 7s 155ms/step - loss: 0.0886 - accuracy: 0.9615
Epoch 17/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0910 - accuracy: 0.9615
Epoch 18/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0909 - accuracy: 0.9602
Epoch 19/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0875 - accuracy: 0.9584
Epoch 20/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0795 - accuracy: 0.9660
Epoch 21/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0907 - accuracy: 0.9627
Epoch 22/100
48/48 [==============================] - 7s 154ms/step - loss: 0.0899 - accuracy: 0.9606
Epoch 23/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0824 - accuracy: 0.9652
Epoch 24/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0870 - accuracy: 0.9618
Epoch 25/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1439 - accuracy: 

...

Epoch 32/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0866 - accuracy: 0.9665
Epoch 33/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0751 - accuracy: 0.9742
Epoch 34/100
48/48 [==============================] - 7s 155ms/step - loss: 0.2027 - accuracy: 0.9316
Epoch 35/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1378 - accuracy: 0.9491
Epoch 36/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0754 - accuracy: 0.9728
Epoch 37/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0597 - accuracy: 0.9762
Epoch 38/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0500 - accuracy: 0.9799
Epoch 39/100
48/48 [==============================] - 7s 156ms/step - loss: 0.1092 - accuracy: 0.9645
Epoch 40/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0715 - accuracy: 0.9725
Epoch 41/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0512

48/48 [==============================] - 7s 156ms/step - loss: 0.0477 - accuracy: 0.9821
Epoch 11/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0426 - accuracy: 0.9836
Epoch 12/100
48/48 [==============================] - 7s 156ms/step - loss: 0.0456 - accuracy: 0.9821
Epoch 13/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0390 - accuracy: 0.9846
Epoch 14/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0422 - accuracy: 0.9827
Epoch 15/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0416 - accuracy: 0.9851
Epoch 16/100
48/48 [==============================] - 7s 156ms/step - loss: 0.0401 - accuracy: 0.9839
Epoch 17/100
48/48 [==============================] - 7s 156ms/step - loss: 0.0457 - accuracy: 0.9832
Epoch 18/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0566 - accuracy: 0.9772
Epoch 19/100
48/48 [==============================] - 7s 156ms/step - loss: 0.0519 - accuracy: 

48/48 [==============================] - 7s 156ms/step - loss: 0.0371 - accuracy: 0.9873
Epoch 91/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1383 - accuracy: 0.9523
Epoch 92/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0488 - accuracy: 0.9811
Epoch 93/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0349 - accuracy: 0.9865
Epoch 94/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0445 - accuracy: 0.9825
Epoch 95/100
48/48 [==============================] - 7s 156ms/step - loss: 0.0433 - accuracy: 0.9835
Epoch 96/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0379 - accuracy: 0.9870
Epoch 97/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0765 - accuracy: 0.9719
Epoch 98/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0399 - accuracy: 0.9843
Epoch 99/100
48/48 [==============================] - 8s 156ms/step - loss: 0.0430 - accuracy: 

Epoch 69/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0338 - accuracy: 0.9865
Epoch 70/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0337 - accuracy: 0.9869
Epoch 71/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0343 - accuracy: 0.9878
Epoch 72/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0878 - accuracy: 0.9762
Epoch 73/100
48/48 [==============================] - 7s 155ms/step - loss: 0.1457 - accuracy: 0.9519
Epoch 74/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0870 - accuracy: 0.9699
Epoch 75/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0535 - accuracy: 0.9802
Epoch 76/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0590 - accuracy: 0.9780
Epoch 77/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0437 - accuracy: 0.9845
Epoch 78/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0443

48/48 [==============================] - 7s 155ms/step - loss: 0.0341 - accuracy: 0.9874
Epoch 48/100
48/48 [==============================] - 7s 156ms/step - loss: 0.0430 - accuracy: 0.9857
Epoch 49/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0409 - accuracy: 0.9848
Epoch 50/100
48/48 [==============================] - 7s 156ms/step - loss: 0.0329 - accuracy: 0.9865
Epoch 51/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0829 - accuracy: 0.9730
Epoch 52/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0521 - accuracy: 0.9792
Epoch 53/100
48/48 [==============================] - 8s 156ms/step - loss: 0.0379 - accuracy: 0.9865
Epoch 54/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0303 - accuracy: 0.9873
Epoch 55/100
48/48 [==============================] - 7s 155ms/step - loss: 0.0312 - accuracy: 0.9885
Epoch 56/100
48/48 [==============================] - 8s 158ms/step - loss: 0.0468 - accuracy: 

In [ ]:
np.average(accuracies)

0.9090433482810164

In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir_name,
    target_size=img_size,
    color_mode='rgba',
    batch_size=batch_size,
    class_mode='categorical')

Found 3344 images belonging to 2 classes.


In [ ]:
loss, acc = model.evaluate_generator(test_generator)
acc

0.7054426074028015